#### Roughly following the guide in:
###### https://pangkh98.medium.com/multi-step-multivariate-time-series-forecasting-using-lstm-92c6d22cd9c2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

features = ['SWTP Total Influent Flow', 'SWTP Plant 1 Influent Flow', 'SWTP Plant 2 Influent Flow',
            'Wilsons Gauge Height (ft)', 'James Gauge Height (ft)', 
            'Fire 120 Hour Rainfall Aggregate', 'Bingham 120 Hour Rainfall Aggregate', 'Field 120 Hour Rainfall Aggregate', 
            'Springfield Plateau Aquifer Depth to Water Level (ft)', 'Ozark Aquifer Depth to Water Level (ft)']

# features = ["Springfield Plateau Aquifer Depth to Water Level (ft)", "Fire 120 Hour Rainfall Aggregate", "SWTP Total Influent Flow"]
# dataset = pd.read_csv("Train and Test Data.csv", usecols=features)
dataset = pd.read_csv("Imputed Data.csv", usecols=features)
arr = np.array(dataset["SWTP Total Influent Flow"])
dataset["Target"] = arr         # adding another influent flow feature so that past values can be used to predict future values
values = dataset.values

# linear transformation of each feature from [min, max] to [0, 1]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)

In [ ]:
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras_tuner.tuners import BayesianOptimization
import os

# path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting\\keras_tuner_attempt4\\model"

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# for testing on different parts of the total year testing set
# use slide = 15 for 8 different windows will cover 24 days of 72 hour forecasts
def sliding_window(X, y, n_test, slide):
    split_point = X.shape[0] - n_test + slide
    train_X , train_y = X[:split_point, :] , y[:split_point, :]
    test_X , test_y = X[split_point:, :] , y[split_point:, :]
    return train_X, train_y, test_X, test_y

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value = 40, max_value = 200, step = 5), 
               activation = 'tanh', return_sequences = True, input_shape = (n_steps_in, len(features))))
    model.add(LSTM(units = hp.Int('units', min_value = 40, max_value = 200, step = 5)))
    model.add(Dense(24))   # for predicting 24 hours -- if desire more, change
    model.add(Activation('linear'))
    model.compile(loss = 'mse', metrics = 'mse', optimizer = keras.optimizers.Adam(
        hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 1e-5])))
    return model

def invNormalize(arr, minimum, maximum):
    return (maximum - minimum) * arr + minimum

def predict(model, test_X):
    # to be able to inverse scale predictions
    df = pd.read_csv("Train and Test Data.csv", usecols=["SWTP Total Influent Flow"])
    arr = np.array(df["SWTP Total Influent Flow"])
    maximum = np.max(arr)
    minimum = np.min(arr)

    #predictions and rescaling to [min, max]
    y_pred = model.predict(test_X)
    y_pred_inv = np.array([invNormalize(x, minimum, maximum) for x in y_pred])
    test_y_inv = np.array([invNormalize(x, minimum, maximum) for x in test_y])
    print("y_pred_inv:",y_pred_inv.shape)
    print("test_y_inv:",y_pred_inv.shape)
    
    return y_pred_inv, test_y_inv

def mseForecast(y, y_pred):
    # change so is only for a range, i.e. first 72 days of test set
    # mse is gonna scale really badly the further out it goes
    # msut fix! cannot calculate mse from a full year, must use a rolling window
    # that forecasts 3 days in advance, has new next hour put into it, then has next 3 day forecast one hour after
    totalMSE = 0
    for i in range(y.shape[0]):
        totalMSE += mean_squared_error(y[i], y_pred[i])
    avgMSE = totalMSE / y.shape[0]
    print("Total Avg MSE:", avgMSE)
    return avgMSE

def saveResults(path, firstMSE, avgMSE, n_epochs, hours=36):
    txt = "n_steps_in = " + str(hours)
    txt += "\nepochs = " + str(n_epochs)
    txt += "\nFirst 10 Avg MSE: " + str(round(firstMSE, 4))
    txt += "\nTotal Avg MSE: " + str(round(avgMSE, 4))
    txt += "\n\nForm:\nLSTM\nLSTM\nDense(24)\nActivation('linear')"
    with open(path + "\\results.txt", 'w') as f:
        f.write(txt)


In [ ]:
# choose a number of time steps 
n_steps_in = 36
# n_steps_in = 48
n_steps_out = 24

# covert into input/output
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
print ("X.shape" , X.shape)                             # [rows, time lags backward, features]
print ("y.shape" , y.shape)                             # [rows, future time values]


# splitting into training and testing
n_tests = [8760, 6552, 4344, 2280]                      # for different validation windows throughout the year
# epochList = [3, 5, 8, 10, 12, 15, 20, 25, 30]
# epochList = [4, 6, 7, 9, 11, 13, 14]
epochList = [20, 25]

# for i in n_tests:               # number of testing points per year
#     for s in range(7):          # number of sliding windows per testing point
lst = []
for i in range(len(epochList)):
    path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting\\keras_tuner_attempt"
    path += str(i+15)
    modelPath = path + "\\model"
    project_title = "keras_tuner_attempt" + str(i+15)
    n_epochs = epochList[i]
    print("Number epochs:", n_epochs)
    for n in n_tests[:1]:               
        for s in range(1):
            train_X, train_y, test_X, test_y = sliding_window(X, y, n, 15 * s)
            # print("\ntrain_X.shape", train_X.shape)
            # print("train_y.shape", train_y.shape)
            # print("test_X.shape", test_X.shape)
            # print("test_y.shape", test_y.shape)

            # tuning model with keras tuner
            bayesian_opt_tuner = BayesianOptimization(
                build_model,
                objective='mse',
                max_trials=3,
                executions_per_trial=1,
                directory=os.path.normpath('C:/Users/natha/Desktop/Undergrad/Spring2022/MTH 596 PIC Math/Project - Group 2/Project/Forecasting'),
                project_name=project_title,
                overwrite=True)
            bayesian_opt_tuner.search(train_X, train_y, epochs=n_epochs,
                validation_data=(test_X, test_y),
                validation_split=0.2, verbose=1)
            bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)
            model = bayes_opt_model_best_model[0]
            model.save(modelPath)

            string = "Number epochs = " + str(n_epochs)
            with open(path + "\\note.txt", 'w') as f:
                f.write(string)

            # path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting\\keras_tuner_attempt1\\model"
            # model = keras.models.load_model(modelPath)

            # fitting model and predicting
            pred_y_inv, test_y_inv = predict(model, test_X)
            totalMSE = 0
            for k in range(10):
                totalMSE += mean_squared_error(pred_y_inv[k], test_y_inv[k])
            print("First 10 Avg MSE:", totalMSE/10)
            totalAvgMSE = mseForecast(test_y_inv, pred_y_inv)
            saveResults(path, totalMSE/10, totalAvgMSE, n_epochs)

            lst.append((n_epochs, round(totalMSE/10, 4), round(totalAvgMSE, 4)))
print(lst)
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt

# of form (hyperparameter, First MSE, Total Avg MSE) for a LSTM, LSTM, Dense(24), Activation('linear') model with 20 epochs
def weightedMSE(arr):
    results = []
    x1 = 0.3
    A = np.array([[x1, 1-x1]])
    for tup in arr:
        x = np.array([[tup[1]], [tup[2]]])
        b = np.matmul(A, x)
        results.append((tup[0], b[0][0]))
    results.sort(key = lambda x:x[0])
    return results

bestHours = [(12, 158.3091, 81.6040), (18, 63.3076, 94.3174), (24, 44.8115, 99.3262), (30, 39.1650, 92.6211), (36, 36.1072, 85.1316), 
    (42, 48.3943, 110.375), (45, 191.6520, 87.9457), (48, 23.7618, 73.4414), (54, 26.0222, 121.7316), 
    (60, 42.7132, 82.8813), (72, 80.9572, 74.0058)]
# results = weightedMSE(bestHours)

bestEpochs36 = [(5, 27.4284, 74.1717), (10, 22.9803, 67.2062), (15, 57.3269, 77.8016), (30, 56.5516, 148.8478),
    (3, 201.0433, 108.7818), (4, 93.3019, 67.6975), (6, 24.6906, 66.4358), (7, 39.6959, 70.8634), (8, 55.2934, 71.261), 
    (9, 30.4693, 73.3233), (11, 56.3759, 77.3612), (12, 79.1437, 80.6395), (13, 84.1152, 74.4024), (14, 26.3952, 85.5936), 
    (20, 104.5808, 87.708), (25, 78.8238, 86.8419)]
results36 = weightedMSE(bestEpochs36)

bestEpochs48 = [(3, 45.6318, 66.2500), (4, 59.9746, 69.0089), (5, 66.8563, 70.0581), (6, 33.0507, 73.2355), 
    (7, 35.5758, 69.0120), (8, 35.6232, 68.3578), (9, 42.2172, 79.0518), (10, 168.4794, 74.5590), (11, 13.3794, 75.8850),
    (12, 13.7495, 79.9994), (13, 84.1156, 73.6373), (14, 28.7678, 102.5217), (15, 51.4311, 86.8468), 
    (20, 30.0797, 106.8292), (25, 20.7012, 93.9292), (30, 69.5697, 94.9882)]
results48 = weightedMSE(bestEpochs48)

# graphing
for results, label, color in zip([results36, results48], ["36 Hour", "48 Hour"], ["red", "blue"]):
    x = [tup[0] for tup in results]
    y = [tup[1] for tup in results]
    plt.plot(x, y, label=label, color=color)
    plt.scatter(x, y, color="green")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import json

# choose a number of time steps 
# n_steps_in = 36
n_steps_in = 48
n_steps_out = 24

# covert into input/output
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
print ("X.shape" , X.shape)                             # [rows, time lags backward, features]
print ("y.shape" , y.shape)                             # [rows, future time values]


n_tests = [8760, 6552, 4344, 2280]                      # for different validation windows throughout the year
epochList36 = [10, 5, 15, 30, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 20, 25]
epochList48 = [3, 5, 8, 10, 12, 15, 20, 25, 30, 4, 6, 7, 9, 11, 13, 14]
epoch48ValidationsIndicies = [13, 4]
epoch36ValidationsIndicies = [6, 0]
# 48 - 11, 12 epochs best
# 36 - 6, 10 epochs best

epochsToUse = [epochList48[i] for i in epoch48ValidationsIndicies]
for n_epochs, folderIndex in zip(epochsToUse, epoch48ValidationsIndicies):
    path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting\\48 hour epoch tuning\\keras_tuner_attempt"
    path += str(folderIndex+1)
    modelPath = path + "\\model"
    print(path)
    print("Number epochs:", n_epochs)

    testMSE = [[0, 0, 0], [1, 0, 0], [2, 0, 0], [3, 0, 0]]      # n_test period, first 96 hour prediction mse, shifted 480 hours forward 96 hour pred
    for n in range(len(n_tests)):               
        for s in range(2):
            # getting data for validation test
            train_X, train_y, test_X, test_y = sliding_window(X, y, n_tests[n], 480 * s)

            # fitting model and predicting
            model = keras.models.load_model(modelPath)
            model.fit(train_X, train_y, epochs=n_epochs, verbose=1, validation_data=(test_X, test_y), validation_split=0.2)
            pred_y_inv, test_y_inv = predict(model, test_X)
            totalMSE = 0
            numHoursForward = 96
            for k in range(numHoursForward):
                totalMSE += mean_squared_error(pred_y_inv[k], test_y_inv[k])
            # print("First {n} Avg MSE: {mse}".format(n=numHoursForward, mse=totalMSE/numHoursForward))

            testMSE[n][s+1] = totalMSE/numHoursForward

    # saving validation results
    with open(path + "\\validation results.json", "w") as f:
        json.dump(testMSE, f)


In [ ]:
def combineValidationResults(results):
    sum = 0
    for l in results:
        for i in range(1, 3):
            sum += l[i]
    print("MSE = ", sum/8)
    
epoch12Hour48Results = [[0, 13.265085957764384, 5.664549392861499], [1, 322.14254964814444, 89.00012063018733], 
                        [2, 30.94971330178527, 15.442300796366476], [3, 23.369238580467254, 21.40317229597237]]
epoch11Hour48Results = [[0, 18.972753015342423, 5.010667427249979], [1, 275.55779577332555, 133.88350945889738], 
                        [2, 33.828710449148744, 12.727969694411149], [3, 15.03309853350242, 24.127845879349817]]
combineValidationResults(epoch12Hour48Results)
combineValidationResults(epoch11Hour48Results)

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tensorflow import keras
%matplotlib qt

# getting 
n_steps_in = 48
n_steps_out = 24
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
train_X, train_y, test_X, test_y = sliding_window(X, y, 8760, 0)    # a full year of data

modelPath = "48 hour epoch tuning\\keras_tuner_attempt14\\model"
model = model = keras.models.load_model(modelPath)
y_pred_inv, y_test_inv = predict(model, test_X)

x = list(range(len(y_pred_inv)))
y = [mean_squared_error(y_pred_inv[i], y_test_inv[i]) for i in range(len(y_pred_inv))]



# plotting the mse of 24 hour future predictions
df = pd.read_csv("Imputed Data.csv", usecols=["DateTime", "Fire Rainfall (in)", "Fire 168 Hour Rainfall Aggregate"], parse_dates=["DateTime"])
dates = np.array(df["DateTime"])
dates = dates[-1*test_y.shape[0]-24:-24]

fig, ax = plt.subplots()
formatter = mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(), formats=["%Y", "%Y-%b", "%b-%d", "%d %H:%M", "%d %H:%M", "%H:%M"])
locator = mdates.AutoDateLocator()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(locator)
fig.autofmt_xdate()

ax.set_title("RMSE of 24 Hour Predictions")
ax.plot(dates, np.sqrt(y))  # root mean square error
plt.show()

y_pred_inv: (8760, 24)
test_y_inv: (8760, 24)


In [ ]:
# change date to cheek forecasted predictions vs actual values on that date
dateDesired = "2021-10-23T00:00:00.000000000"
index = 0
for i in range(len(dates)):
    if dateDesired == str(dates[i]):
        index = i
        break
print(dates[index])
print(y_pred_inv[index])
print(y_test_inv[index])
fig, ax = plt.subplots()
ax.plot(list(range(24)), y_test_inv[index], label="Actual")
ax.plot(list(range(24)), y_pred_inv[index], label="Prediction")
ax.legend()
ax.set_title("Results for " + dateDesired[:10] + " at " + dateDesired[11:16])
plt.show()

In [ ]:
from dcor import distance_correlation
from scipy.stats import pearsonr

rmseArr = np.sqrt(y)
df = pd.read_csv("Imputed Data.csv")
lst = []
for col in df.columns[1:]:
    arr = np.array(df[col])
    arr = arr[-1*test_y.shape[0]-24:-24]
    correlationValue = distance_correlation(arr, rmseArr)
    # correlationValue, _ = pearsonr(arr, rmseArr)
    lst.append((col, correlationValue))

lst.sort(key = lambda x:x[1])
lst = lst[::-1]
for x in lst:
    print(x)